<a href="https://colab.research.google.com/github/hahaeaezz-arch/chaemin-lee/blob/main/Fruit_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

train = pd.read_csv("/content/drive/MyDrive/딥러닝_과제 모음/과일 이름 맞추기/train.csv") # Corrected to load train.csv
test = pd.read_csv("/content/drive/MyDrive/딥러닝_과제 모음/과일 이름 맞추기/test.csv")

X = train.drop("fruit_name", axis=1)
y = train["fruit_name"]

cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

X = pd.get_dummies(X, columns=cat_cols)
test = pd.get_dummies(test, columns=cat_cols)
test = test.reindex(columns=X.columns, fill_value=0)

# Convert 'avg_price ($)' in test set to numeric, coercing errors
test['avg_price ($)'] = pd.to_numeric(test['avg_price ($)'], errors='coerce')

# Impute missing values with the mean of the column
test['avg_price ($)'].fillna(test['avg_price ($)'].mean(), inplace=True)


scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])
test[num_cols] = scaler.transform(test[num_cols])

le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_onehot = to_categorical(y_encoded)

X_train, X_val, y_train, y_val = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(128, activation="relu", input_shape=(X.shape[1],)))
model.add(Dense(64, activation="relu"))
model.add(Dense(y_onehot.shape[1], activation="softmax"))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

es = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
mc = ModelCheckpoint("best_model.h5", monitor="val_accuracy", save_best_only=True, verbose=0)

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), callbacks=[es, mc], verbose=0)

y_pred = model.predict(test)
y_pred_labels = le.inverse_transform(np.argmax(y_pred, axis=1))

submission = pd.read_csv("/content/drive/MyDrive/딥러닝_과제 모음/과일 이름 맞추기/submission.csv")
submission.dropna(axis=1, inplace=True)
submission["fruit_name"] = y_pred_labels
submission.to_csv("new_submission.csv", index=False)

/tmp/ipython-input-1691926360.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['avg_price ($)'].fillna(test['avg_price ($)'].mean(), inplace=True)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
